# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile, visualise
import btrack
print(btrack.__version__)
import os, glob

0.6.1.dev83


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
# base_dir = '/mnt/DATA/macrohet/'
base_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/macrohet_images/NDXXXX_optimisation/ND0000__2023-08-04T15_27_41-Measurement 1/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/macrohet_images/NDXXXX_optimisation/ND0000__2023-08-04T15_27_41-Measurement 1/'
# metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata_fn = os.path.join(base_dir, 'Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/243648 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0203K1F1P1R1,Ok,r02c03f01p01-ch1sk1fk1fl1.tiff,2,3,1,1,0,1,1,...,0,0.135205805,0,2023-08-04T15:28:16.5+01:00,561,599,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
1,0203K1F1P1R2,Ok,r02c03f01p01-ch2sk1fk1fl1.tiff,2,3,1,1,0,2,1,...,0,0.135205805,0,2023-08-04T15:28:16.5+01:00,740,0,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
2,0203K1F1P1R3,Ok,r02c03f01p01-ch3sk1fk1fl1.tiff,2,3,1,1,0,3,1,...,0,0.135205805,0,2023-08-04T15:28:16.767+01:00,640,706,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
3,0203K1F1P2R1,Ok,r02c03f01p02-ch1sk1fk1fl1.tiff,2,3,1,2,0,1,1,...,2E-06,0.135207802,0,2023-08-04T15:28:17.047+01:00,561,599,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
4,0203K1F1P2R2,Ok,r02c03f01p02-ch2sk1fk1fl1.tiff,2,3,1,2,0,2,1,...,2E-06,0.135207802,0,2023-08-04T15:28:17.047+01:00,740,0,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243643,0310K376F9P2R2,Ok,r03c10f09p02-ch2sk376fk1fl1.tiff,3,10,9,2,375,2,1,...,2E-06,0.135064006,337503.973,2023-08-08T13:15:47.533+01:00,740,0,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
243644,0310K376F9P2R3,Ok,r03c10f09p02-ch3sk376fk1fl1.tiff,3,10,9,2,375,3,1,...,2E-06,0.135064006,337503.973,2023-08-08T13:15:47.8+01:00,640,706,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
243645,0310K376F9P3R1,Ok,r03c10f09p03-ch1sk376fk1fl1.tiff,3,10,9,3,375,1,1,...,4E-06,0.135066003,337503.973,2023-08-08T13:15:48.08+01:00,561,599,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."
243646,0310K376F9P3R2,Ok,r03c10f09p03-ch2sk376fk1fl1.tiff,3,10,9,3,375,2,1,...,4E-06,0.135066003,337503.973,2023-08-08T13:15:48.097+01:00,740,0,40,1.1,0.2,"[[0.994928,0,0,15.1],[0,-0.994928,0,-5.3],[0,0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
# metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
metadata_path = glob.glob(os.path.join(base_dir, 'Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
# TEMP FIX FOR BUG IN ASSAY LAYOUT READER - ends up displaying more rows cols then there are so
assay_layout = assay_layout.loc[[idx for idx in assay_layout.index if str(idx[0]) in list(metadata['Row'].unique()) and str(idx[1]) in list(metadata['Col'].unique())]]
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Cell Count double Staining Infection
2 3          NaN  50000    False     False
  4          NaN  50000    False      True
  9          NaN  50000     True     False
  10         NaN  50000     True      True
3 3          NaN  25000    False     False
  4          NaN  25000    False      True
  9          NaN  25000     True     False
  10         NaN  25000     True      True

### Define row and column of choice

In [4]:
row = 3
column = 10

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [5]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
                             # set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

CPU times: user 2.09 s, sys: 44.6 ms, total: 2.13 s
Wall time: 2.3 s


In [91]:
images

dask.array<astype, shape=(376, 3, 6048, 6048), dtype=uint16, chunksize=(1, 3, 2016, 2016), chunktype=numpy.ndarray>

In [6]:
%%time
images = images.compute().compute()

Exception: [Errno 5] Input/output error 
 Could not load file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/macrohet_images/NDXXXX_optimisation/ND0000__2023-08-04T15_27_41-Measurement 1/Images/r03c10f09p01-ch2sk145fk1fl1.tiff

In [7]:
images.shape

(376, 3, 6048, 6048)

# Load tracks

In [21]:
tracks_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/06/07 01:09:01 pm] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
[INFO][2023/06/07 01:09:01 pm] Loading tracks/obj_type_1
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Loading tracks/obj_type_1
[INFO][2023/06/07 01:09:01 pm] Loading LBEP/obj_type_1
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Loading LBEP/obj_type_1
[INFO][2023/06/07 01:09:01 pm] Loading objects/obj_type_1 (46507, 5) (46507 filtered: None)
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Loading objects/obj_type_1 (46507, 5) (46507 filtered: None)
[INFO][2023/06/07 01:09:11 pm] Loading segmentation (75, 6048, 6048)
07-Jun-23 13:09:11 - btrack.io.hdf - INFO     - Loading segmentation (75, 6048, 6048)
[INFO][2023/06/07 01:09:11 pm] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5
07-Jun-23 13:09:11 - btrack.io.hdf - INFO     - Clo

In [59]:
filtered_tracks = [t for t in tracks if len(t) > 25]

In [61]:
napari_filtered_tracks, properties, graph = btrack.utils.tracks_to_napari(filtered_tracks, 
                                                                    ndim=2)

### Recolour tracks

In [41]:
col_segmentation = btrack.utils.update_segmentation(segmentation, filtered_tracks, scale = (5.04, 5.04))

# Launch napari image viewer

In [93]:
%time
viewer = napari.Viewer(title = '3,10 sum projection')

# viewer.add_image(images, 
#                  channel_axis=1,
#                  name=["macrophage", "mtb"],
#                  colormap=["green",  "magenta"],
# #                  contrast_limits=[[100, 6000], [100, 2000]],
#                  contrast_limits=[[0,450], [0,450]], 
#                  visible = True
#                  )
viewer.add_image(images, 
                 channel_axis=1,
#                  name=["macrophage", "mtb"],
#                  colormap=["green",  "magenta"],
# #                  contrast_limits=[[100, 6000], [100, 2000]],
#                  contrast_limits=[[0,450], [0,450]], 
                 visible = True
                 )
# viewer.add_labels(segmentation, 
# #                   num_colors= 1,
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                   name = 'segmentation'
#                  )
# viewer.add_labels(col_segmentation, 
#                   name = 'recolored segmentation'
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                  )
# viewer.add_tracks(napari_tracks, scale = (1,5.04,5.04)
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Properly downscaled tracks", 
# #                     blending="translucent",
# #                     visible=True,
# # #                     scale = (100,1,1)
#                  )
# viewer.add_tracks(napari_tracks, scale = (1,5.04,5.04),
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Properly downscaled tracks", 
# #                     blending="translucent",
# #                     visible=True,
# # #                     scale = (100,1,1)
#                  )

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 38.9 µs


[<Image layer 'Image' at 0x7f1fad5f1970>,
 <Image layer 'Image [1]' at 0x7f207d2ee880>,
 <Image layer 'Image [2]' at 0x7f1f924f0880>]

Rendering frames...


  4%|██                                                   | 4/100 [02:21<1:02:44, 39.22s/it]

In [25]:
visualise.highlight_cell(1251, viewer, tracks)

<Points layer 'cell 1251' at 0x7f235d482790>

In [62]:
viewer.add_tracks(napari_filtered_tracks, scale = (1,5.04,5.04),)

<Tracks layer 'napari_filtered_tracks [2]' at 0x7ff4e5a32430>

In [25]:
viewer.layers['napari_filtered_tracks'].scale = (30, 5.04, 5.04)

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/action_manager.py", line 227, in <lambda>
    button.clicked.connect(lambda: self.trigger(name))
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/action_manager.py", line 427, in trigger
    return self._actions[name].injected()
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/in_n_out/_store.py", line 773, in _exec
    result = func(**{**_kwargs, **bound.arguments})
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/image/_image_key_bindings.py", line 33, in orient_plane_normal_along_x
    orient_plane_normal_around_cursor(layer, plane_normal=(0, 0, 1))
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/utils/interactivity_utils.py", line 126, in orient_plane_normal_around_cursor
    view_direction = layer._world_to_displayed_data_ray(
  File "/home/dayn/minic

In [32]:
viewer.layers['napari_filtered_tracks'].axis_labels=["time", "y", "x"]

In [29]:
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
                    name="hacky downscaled tracks", 
                    blending="translucent",
                    visible=True,
#                     scale = (1,1,1)
                )
# viewer.add_labels(ds_segmentation, 
#                  name = 'downscaled segmentation')

viewer.add_labels(ds_col_segmentation, 
                  name = 'downscaled coloured segmentation',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

viewer.add_labels(ds_col_segmentation_new_tracks, 
                  name = 'downscaled coloured segmentation new tracks',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

<Labels layer 'downscaled coloured segmentation new tracks' at 0x7f15c273ca30>